In [2]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_PATH = "/content/drive/MyDrive/datasets/"
SELECTED_ZIP_FILENAME = "fashion_mnist.zip"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# ============================================================
# NOTEBOOK: Clasificación de IMÁGENES + CNN
# Robustez: split por archivos, class_weight automático, OOM-safe (T4),
#           persistencia (weights.best.keras)
#
# ENTRADA:
#   1) Subes un ZIP manualmente al runtime de Colab (/content/*.zip)
#   2) Estructura interna del ZIP (recomendado):
#        <raiz>/
#          clase_0/   (jpg/png/...)
#          clase_1/
#          ...
#
# SALIDAS:
#   - weights.best.keras
# ============================================================

# =========================
# CELDA 0 — CONFIG GLOBAL + ZIP → WORKDIR + autodetección DATA_DIR
# Supuesto: SIEMPRE subes un .zip manualmente al runtime (/content)
# =========================
import os, glob, zipfile, shutil, random, math, time
import numpy as np
import tensorflow as tf

# -------------------------
# 1) CONFIGURACIÓN GENERAL (SOLO IMAGEN)
# -------------------------
WORKDIR = "/content/dataset"
CLEAN_WORKDIR = True

DRIVE_DATA_SOURCE = os.path.join(DRIVE_PATH, 'fashion_mnist')

SEED = 123
TRAIN_FRAC = 0.70
VAL_FRAC   = 0.15
TEST_FRAC  = 0.15

AUTO = True
IMG_SIZE_MANUAL = (128, 128)   # si AUTO=False
BATCH_MANUAL    = 32           # si AUTO=False

# -------------------------
# 2) OPTIMIZACIÓN GPU T4
# -------------------------
USE_MIXED_PRECISION = True
if USE_MIXED_PRECISION:
    try:
        from tensorflow.keras import mixed_precision
        mixed_precision.set_global_policy("mixed_float16")
        print("Mixed precision activada:", mixed_precision.global_policy())
    except Exception as e:
        print("No se pudo activar mixed precision:", e)

AUTOTUNE = tf.data.AUTOTUNE
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)


# -------------------------
# 3) DETECTAR ZIP EN DRIVE Y COPIAR/DESCOMPRIMIR
# -------------------------
# Usar SELECTED_ZIP_FILENAME definido en la celda anterior
zip_name_full_path = os.path.join(DRIVE_DATA_SOURCE, SELECTED_ZIP_FILENAME)
assert os.path.exists(zip_name_full_path), f"No se encontró el archivo ZIP '{SELECTED_ZIP_FILENAME}' en {DRIVE_DATA_SOURCE}."
zip_base_name = os.path.basename(zip_name_full_path)

print("ZIP seleccionado en Drive:", zip_base_name)
print("Última modificación:", time.ctime(os.path.getmtime(zip_name_full_path)))


# -------------------------
# 4) PREPARAR WORKDIR Y DESCOMPRIMIR
# -------------------------
if CLEAN_WORKDIR and os.path.isdir(WORKDIR):
    shutil.rmtree(WORKDIR)
os.makedirs(WORKDIR, exist_ok=True)

print(f"Descomprimiendo {zip_base_name} en {WORKDIR}...")
with zipfile.ZipFile(zip_name_full_path, "r") as z:
    z.extractall(WORKDIR)

print("Dataset descomprimido en:", WORKDIR)
!ls -lah "{WORKDIR}"


# -------------------------
# 5) AUTODETECTAR DATA_DIR REAL
#   buscamos una carpeta que contenga subcarpetas con archivos válidos
# -------------------------
def find_data_root_images(workdir):
    exts = (".jpg",".jpeg",".png",".bmp",".webp")

    candidates = [workdir]
    candidates += [os.path.join(workdir, d) for d in os.listdir(workdir) if os.path.isdir(os.path.join(workdir, d))]

    def score_dir(d):
        if not os.path.isdir(d):
            return -1, 0, 0
        subdirs = [os.path.join(d, s) for s in os.listdir(d) if os.path.isdir(os.path.join(d, s))]
        if not subdirs:
            return -1, 0, 0

        good_folders = 0
        total_files = 0
        for sd in subdirs:
            n = 0
            for ext in exts:
                n += len(glob.glob(os.path.join(sd, f"*{ext}")))
            if n > 0:
                good_folders += 1
                total_files += n
        return good_folders, total_files, len(subdirs)

    best = None
    best_score = (-1, -1, -1)
    for c in candidates:
        sc = score_dir(c)
        if sc > best_score:
            best_score = sc
            best = c

    if best is None or best_score[0] < 2:
        raise ValueError(
            "No pude detectar una raíz de dataset válida.\n"
            "Asegúrate de que el ZIP contenga una carpeta con subcarpetas y archivos de imagen.\n"
            f"WORKDIR={workdir}"
        )

    return best, best_score

DATA_DIR, sc = find_data_root_images(WORKDIR)

print("\nCONFIG FINAL:")
print("  WORKDIR   :", WORKDIR)
print("  DATA_DIR  :", DATA_DIR)
print("  (folders_con_archivos, total_archivos, subcarpetas):", sc)
print("  GPU       :", tf.config.list_physical_devices("GPU"))


Mixed precision activada: <DTypePolicy "mixed_float16">
ZIP seleccionado en Drive: fashion_mnist.zip
Última modificación: Mon Jan 12 21:42:45 2026
Descomprimiendo fashion_mnist.zip en /content/dataset...
Dataset descomprimido en: /content/dataset
total 2.2M
drwxr-xr-x 12 root root 4.0K Jan 21 07:20 .
drwxr-xr-x  1 root root 4.0K Jan 21 07:19 ..
drwxr-xr-x  2 root root 216K Jan 21 07:20 0
drwxr-xr-x  2 root root 228K Jan 21 07:20 1
drwxr-xr-x  2 root root 204K Jan 21 07:20 2
drwxr-xr-x  2 root root 204K Jan 21 07:20 3
drwxr-xr-x  2 root root 216K Jan 21 07:20 4
drwxr-xr-x  2 root root 220K Jan 21 07:20 5
drwxr-xr-x  2 root root 224K Jan 21 07:19 6
drwxr-xr-x  2 root root 220K Jan 21 07:20 7
drwxr-xr-x  2 root root 200K Jan 21 07:20 8
drwxr-xr-x  2 root root 200K Jan 21 07:20 9

CONFIG FINAL:
  WORKDIR   : /content/dataset
  DATA_DIR  : /content/dataset
  (folders_con_archivos, total_archivos, subcarpetas): (10, 70000, 10)
  GPU       : [PhysicalDevice(name='/physical_device:GPU:0', devi

In [5]:
# ======================================
# CELDA 1 — UTILIDADES: CLASES + SPLIT + DESBALANCE + VALIDACIÓN FUERTE (IMAGEN)
# ======================================
import numpy as np
import os, glob

def list_class_folders(data_dir):
    classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
    if len(classes) < 2:
        raise ValueError(f"Se requieren >=2 subcarpetas en: {data_dir}. Encontré: {classes}")
    return classes

def list_files_by_class_image(data_dir):
    classes = list_class_folders(data_dir)
    exts = (".jpg",".jpeg",".png",".bmp",".webp")

    files = []
    labels = []
    per_class = []

    for i, cls in enumerate(classes):
        cls_dir = os.path.join(data_dir, cls)
        cls_files = []
        for ext in exts:
            cls_files.extend(glob.glob(os.path.join(cls_dir, f"*{ext}")))
        cls_files = sorted(cls_files)

        per_class.append((cls, len(cls_files)))
        files.extend(cls_files)
        labels.extend([i]*len(cls_files))

    return classes, np.array(files), np.array(labels, dtype=np.int32), per_class

def stratified_split(files, labels, train_frac, val_frac, test_frac, seed=123):
    assert abs(train_frac + val_frac + test_frac - 1.0) < 1e-9
    rng = np.random.default_rng(seed)
    idx = np.arange(len(files))

    train_idx, val_idx, test_idx = [], [], []
    for c in np.unique(labels):
        c_idx = idx[labels == c]
        rng.shuffle(c_idx)
        n = len(c_idx)
        if n == 0:
            continue

        n_train = int(round(n * train_frac))
        n_val   = int(round(n * val_frac))

        n_train = max(1, min(n_train, n))
        n_val = min(n_val, n - n_train)

        train_idx.extend(c_idx[:n_train])
        val_idx.extend(c_idx[n_train:n_train+n_val])
        test_idx.extend(c_idx[n_train+n_val:])

    rng.shuffle(train_idx); rng.shuffle(val_idx); rng.shuffle(test_idx)
    return np.array(train_idx), np.array(val_idx), np.array(test_idx)

def compute_class_weight(train_labels, num_classes):
    counts = np.bincount(train_labels, minlength=num_classes).astype(np.int64)
    N = counts.sum()
    weights = {}
    for c in range(num_classes):
        weights[c] = 0.0 if counts[c] == 0 else float(N) / float(num_classes * counts[c])
    return counts, weights

# -------------------------
# CARGA (IMAGEN)
# -------------------------
classes, all_files, all_labels, per_class = list_files_by_class_image(DATA_DIR)
num_classes = len(classes)

train_idx, val_idx, test_idx = stratified_split(all_files, all_labels, TRAIN_FRAC, VAL_FRAC, TEST_FRAC, SEED)

# -------------------------
# PRINTS + VALIDACIÓN FUERTE
# -------------------------
print("DATA_DIR:", DATA_DIR)
print("Num clases:", num_classes)
print("Total ejemplos:", len(all_files))
print("\nConteo por clase (primeras 20):")
for cls, n in per_class[:20]:
    print(f"  {cls:<30s} {n}")
if len(per_class) > 20:
    print("  ...")

if len(all_files) == 0:
    raise ValueError(
        "No se encontró ningún archivo de imagen válido en las carpetas.\n"
        f"DATA_DIR={DATA_DIR}"
    )

print("\nSplit tamaños:", "train", len(train_idx), "| val", len(val_idx), "| test", len(test_idx))
if len(train_idx) == 0 or len(val_idx) == 0 or len(test_idx) == 0:
    raise ValueError(
        "Alguno de los splits quedó vacío. Revisa que haya suficientes ejemplos.\n"
        f"train={len(train_idx)}, val={len(val_idx)}, test={len(test_idx)}"
    )

# Desbalance (usa TRAIN)
train_labels = all_labels[train_idx]
class_counts, class_weight = compute_class_weight(train_labels, num_classes)

min_count = int(class_counts.min()) if len(class_counts) else 0
max_count = int(class_counts.max()) if len(class_counts) else 0
imbalance_ratio = (max_count / min_count) if (min_count > 0) else float("inf")

IMBALANCED = (imbalance_ratio >= 2.0) or (min_count <= 10)

TINY_CLASS_THRESHOLD = 5
RARE_CLASS_THRESHOLD = 10

tiny_idx = np.where(class_counts <= TINY_CLASS_THRESHOLD)[0]
rare_idx = np.where((class_counts > TINY_CLASS_THRESHOLD) & (class_counts <= RARE_CLASS_THRESHOLD))[0]
zero_idx = np.where(class_counts == 0)[0]

HAS_TINY_CLASSES = len(tiny_idx) > 0
HAS_RARE_CLASSES = len(rare_idx) > 0

USE_CLASS_WEIGHT = IMBALANCED or HAS_TINY_CLASSES or HAS_RARE_CLASSES
MONITOR_METRIC = "val_loss" if (IMBALANCED or HAS_TINY_CLASSES) else "val_accuracy"

print("\nDistribución TRAIN: min", min_count, "| max", max_count, "| ratio", imbalance_ratio)
print("IMBALANCED:", IMBALANCED)
print("HAS_TINY_CLASSES:", HAS_TINY_CLASSES, "| HAS_RARE_CLASSES:", HAS_RARE_CLASSES)
print("USE_CLASS_WEIGHT:", USE_CLASS_WEIGHT)
print("MONITOR_METRIC:", MONITOR_METRIC)

print("\n=== SANITY CHECK SPLITS ===")
print("Labels min/max:", int(all_labels.min()), int(all_labels.max()))
print("Num clases declarado:", num_classes)

def bincountK(y, K):
    return np.bincount(y, minlength=K)

print("Train per class:", bincountK(all_labels[train_idx], num_classes).tolist())
print("Val   per class:", bincountK(all_labels[val_idx],   num_classes).tolist())
print("Test  per class:", bincountK(all_labels[test_idx],  num_classes).tolist())


DATA_DIR: /content/dataset
Num clases: 10
Total ejemplos: 70000

Conteo por clase (primeras 20):
  0                              7000
  1                              7000
  2                              7000
  3                              7000
  4                              7000
  5                              7000
  6                              7000
  7                              7000
  8                              7000
  9                              7000

Split tamaños: train 49000 | val 10500 | test 10500

Distribución TRAIN: min 4900 | max 4900 | ratio 1.0
IMBALANCED: False
HAS_TINY_CLASSES: False | HAS_RARE_CLASSES: False
USE_CLASS_WEIGHT: False
MONITOR_METRIC: val_accuracy

=== SANITY CHECK SPLITS ===
Labels min/max: 0 9
Num clases declarado: 10
Train per class: [4900, 4900, 4900, 4900, 4900, 4900, 4900, 4900, 4900, 4900]
Val   per class: [1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050]
Test  per class: [1050, 1050, 1050, 1050, 1050, 1050, 1050, 1050, 

In [6]:
# ==========================================================
# CELDA 2 — AUTO-CONFIG (IMG_SIZE + BATCH) + CHANNELS (IMAGEN)
# ==========================================================
import math
import PIL.Image

def choose_img_size_and_batch(sample_shape, t4=True):
    H, W, C = sample_shape
    m = min(H, W)
    if m <= 32:
        img = (32, 32);  batch = 256
    elif m <= 96:
        img = (96, 96);  batch = 64
    else:
        img = (128, 128); batch = 32
    return img, batch, C

# ---- Inferir shape/canales con una muestra ----
p0 = all_files[0]
im = np.array(PIL.Image.open(p0))
if im.ndim == 2:
    H, W = im.shape
    C = 1
else:
    H, W, C = im.shape
sample_shape = (H, W, C)

if AUTO:
    IMG_SIZE, BATCH, CHANNELS = choose_img_size_and_batch(sample_shape, t4=True)
else:
    IMG_SIZE = IMG_SIZE_MANUAL
    BATCH = BATCH_MANUAL
    CHANNELS = 3  # si tus imágenes son RGB; si son grises, cambia a 1

print("AUTO:", AUTO)
print("IMG_SIZE:", IMG_SIZE)
print("BATCH:", BATCH)
print("CHANNELS:", CHANNELS)


AUTO: True
IMG_SIZE: (32, 32)
BATCH: 256
CHANNELS: 1


In [7]:
# ==========================================================
# CELDA 3 — PIPELINE IMAGEN (tf.data) + build_datasets(batch)
# ==========================================================
import tensorflow as tf

def decode_image(path, label, img_size, channels):
    img = tf.io.read_file(path)
    img = tf.io.decode_image(img, channels=channels, expand_animations=False)
    img = tf.image.resize(img, img_size, antialias=True)
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

def make_image_ds(files, labels, img_size, channels, batch, training=False, seed=123):
    ds = tf.data.Dataset.from_tensor_slices((files, labels))
    if training:
        ds = ds.shuffle(len(files), seed=seed, reshuffle_each_iteration=True)
    ds = ds.map(lambda p,y: decode_image(p,y,img_size,channels), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch).prefetch(AUTOTUNE)
    return ds

def build_datasets(batch):
    train_ds = make_image_ds(all_files[train_idx], all_labels[train_idx], IMG_SIZE, CHANNELS, batch, training=True, seed=SEED)
    val_ds   = make_image_ds(all_files[val_idx],   all_labels[val_idx],   IMG_SIZE, CHANNELS, batch, training=False)
    test_ds  = make_image_ds(all_files[test_idx],  all_labels[test_idx],  IMG_SIZE, CHANNELS, batch, training=False)
    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = build_datasets(BATCH)
print("Datasets listos (imagen). BATCH =", BATCH)


Datasets listos (imagen). BATCH = 256


In [8]:
# ==========================================================
# CELDA 5 — AUGMENT ROBUSTO (IMAGEN)
# ==========================================================
from tensorflow.keras import layers

augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.10),
], name="augment_image")


In [9]:
# ==========================================================
# CELDA 6 — MODELO CNN ROBUSTO (GAP + BN + Dropout)
# ==========================================================
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow as tf

def build_cnn(img_size, channels, num_classes, augment_layer):
    inputs = layers.Input(shape=(img_size[0], img_size[1], channels))
    x = augment_layer(inputs)

    x = layers.Conv2D(32, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Conv2D(128, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D()(x)

    x = layers.Dropout(0.25)(x)
    x = layers.GlobalAveragePooling2D()(x)

    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.30)(x)

    outputs = layers.Dense(num_classes, activation="softmax", dtype="float32")(x)
    return models.Model(inputs, outputs)

model = build_cnn(IMG_SIZE, CHANNELS, num_classes, augment)

metrics = ["accuracy"]
if num_classes >= 10:
    metrics.append(tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name="top5_acc"))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=metrics
)

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ augment_image (Sequential)      │ (None, 32, 32, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 129,162 (504.54 KB)

 Trainable params: 128,714 (502.79 KB)

 Non-trainable params: 448 (1.75 KB)

In [10]:
# ==========================================================
# CELDA 7 — TRAIN (OOM-safe + class_weight + política robusta)
# ==========================================================
import gc
import tensorflow as tf
import numpy as np

if HAS_TINY_CLASSES:
    LR = 5e-4
    PATIENCE = 8
else:
    LR = 1e-3
    PATIENCE = 5

try:
    model.optimizer.learning_rate.assign(LR)
except Exception:
    model.optimizer.learning_rate = LR

print("LR usado:", LR)
print("PATIENCE usado:", PATIENCE)
print("MONITOR_METRIC:", MONITOR_METRIC)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor=MONITOR_METRIC,
        patience=PATIENCE,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath="/content/weights.best.keras",
        monitor=MONITOR_METRIC,
        save_best_only=True
    )
]

fit_class_weight = class_weight if USE_CLASS_WEIGHT else None

def batch_candidates(b0):
    cands = [int(b0)]
    while cands[-1] > 8:
        cands.append(cands[-1] // 2)
    cands = sorted(set([b for b in cands if b >= 8]), reverse=True)
    return cands

BATCH_TRIES = batch_candidates(BATCH)
print("BATCH tries:", BATCH_TRIES)

history = None
last_err = None

for b_try in BATCH_TRIES:
    try:
        train_ds, val_ds, test_ds = build_datasets(b_try)

        print(f"\nEntrenando con BATCH={b_try} | monitor={MONITOR_METRIC} | class_weight={USE_CLASS_WEIGHT}")
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=30,
            callbacks=callbacks,
            class_weight=fit_class_weight
        )
        BATCH = b_try
        last_err = None
        break

    except tf.errors.ResourceExhaustedError as e:
        last_err = e
        print(f"\n⚠️ OOM con BATCH={b_try}. Reintentando con batch menor...")
        try:
            del train_ds, val_ds, test_ds
        except Exception:
            pass
        gc.collect()

if history is None and last_err is not None:
    raise last_err

print("\n✅ Entrenamiento finalizado. BATCH final usado:", BATCH)

# -------------------------
# INFORME DE EPOCH FINAL (REAL)
# -------------------------
hist = history.history
mon = MONITOR_METRIC

if mon in hist:
    if "acc" in mon:
        best_epoch = int(np.argmax(hist[mon]) + 1)
        best_value = float(np.max(hist[mon]))
        mode = "max"
    else:
        best_epoch = int(np.argmin(hist[mon]) + 1)
        best_value = float(np.min(hist[mon]))
        mode = "min"

    print("\n📌 RESUMEN DE ENTRENAMIENTO")
    print(f"Monitor usado      : {mon} ({mode})")
    print(f"Epoch seleccionado : {best_epoch}")
    print(f"Mejor {mon}        : {best_value:.4f}")
    print("✔ restore_best_weights=True → el modelo en memoria quedó en ese epoch")
else:
    print("\n⚠️ No se pudo determinar el epoch final (monitor no encontrado).")
    print("Keys disponibles:", list(hist.keys()))


LR usado: 0.001
PATIENCE usado: 5
MONITOR_METRIC: val_accuracy
BATCH tries: [256, 128, 64, 32, 16, 8]

Entrenando con BATCH=256 | monitor=val_accuracy | class_weight=False
Epoch 1/30
192/192 ━━━━━━━━━━━━━━━━━━━━ 30s 96ms/step - accuracy: 0.5892 - loss: 1.1519 - top5_acc: 0.9359 - val_accuracy: 0.1677 - val_loss: 3.9466 - val_top5_acc: 0.5000
Epoch 2/30
192/192 ━━━━━━━━━━━━━━━━━━━━ 16s 83ms/step - accuracy: 0.7957 - loss: 0.5647 - top5_acc: 0.9947 - val_accuracy: 0.2671 - val_loss: 2.2725 - val_top5_acc: 0.7455
Epoch 3/30
192/192 ━━━━━━━━━━━━━━━━━━━━ 17s 89ms/step - accuracy: 0.8199 - loss: 0.4932 - top5_acc: 0.9955 - val_accuracy: 0.7907 - val_loss: 0.5732 - val_top5_acc: 0.9946
Epoch 4/30
192/192 ━━━━━━━━━━━━━━━━━━━━ 16s 84ms/step - accuracy: 0.8356 - loss: 0.4503 - top5_acc: 0.9970 - val_accuracy: 0.8389 - val_loss: 0.4263 - val_top5_acc: 0.9962
Epoch 5/30
192/192 ━━━━━━━━━━━━━━━━━━━━ 16s 81ms/step - accuracy: 0.8442 - loss: 0.4286 - top5_acc: 0.9967 - val_accuracy: 0.7925 - val_loss

In [11]:
# ==========================================================
# CELDA 7.5 — DIAGNÓSTICO AUTOMÁTICO DEL ENTRENAMIENTO (mejorado)
# ==========================================================
import numpy as np

def diagnose_training_v2(history, num_classes, monitor_metric="val_loss", patience=None):
    h = history.history
    epochs_ran = len(next(iter(h.values()))) if len(h) else 0

    def arr(key):
        v = h.get(key, None)
        return None if v is None else np.array(v, dtype=float)

    acc   = arr("accuracy")
    vacc  = arr("val_accuracy")
    loss  = arr("loss")
    vloss = arr("val_loss")

    chance = 1.0 / float(num_classes) if num_classes else np.nan

    mon = arr(monitor_metric)
    if mon is None:
        print("⚠️ No existe monitor_metric en history:", monitor_metric)
        print("Keys:", list(h.keys()))
        return

    if "acc" in monitor_metric:
        best_i = int(np.nanargmax(mon))
        best_val = float(np.nanmax(mon))
        mode = "max"
    else:
        best_i = int(np.nanargmin(mon))
        best_val = float(np.nanmin(mon))
        mode = "min"

    def safe_get(a, i):
        return float(a[i]) if a is not None and len(a) > i else np.nan

    last_i = epochs_ran - 1

    last_acc  = safe_get(acc, last_i)
    last_vacc = safe_get(vacc, last_i)
    last_loss = safe_get(loss, last_i)
    last_vloss= safe_get(vloss, last_i)

    best_acc  = safe_get(acc, best_i)
    best_vacc = safe_get(vacc, best_i)
    best_loss = safe_get(loss, best_i)
    best_vloss= safe_get(vloss, best_i)

    degrade_loss = (not np.isnan(best_vloss) and not np.isnan(last_vloss) and last_vloss > best_vloss * 1.15)
    degrade_acc  = (not np.isnan(best_vacc) and not np.isnan(last_vacc) and last_vacc < best_vacc - 0.07)

    gap_best = best_acc - best_vacc if (not np.isnan(best_acc) and not np.isnan(best_vacc)) else np.nan
    gap_last = last_acc - last_vacc if (not np.isnan(last_acc) and not np.isnan(last_vacc)) else np.nan

    def slope(a):
        if a is None or len(a) < 6:
            return np.nan
        y = a[-5:]
        x = np.arange(len(y), dtype=float)
        return float(np.polyfit(x, y, 1)[0])

    s_acc  = slope(acc)
    s_vacc = slope(vacc)
    s_loss = slope(loss)
    s_vloss= slope(vloss)

    print("\n" + "="*60)
    print("DIAGNÓSTICO 7.5 — RESUMEN (v2)")
    print("="*60)
    print(f"Clases: {num_classes} | azar≈ {chance:.4f} | epochs corridos: {epochs_ran}")
    print(f"Monitor: {monitor_metric} ({mode}) | best_epoch={best_i+1} | best={best_val:.4f}")
    if patience is not None:
        print(f"Patience: {patience}")

    print("\n— En BEST epoch (lo que queda en memoria si restore_best_weights=True) —")
    print(f"  acc={best_acc:.4f} | val_acc={best_vacc:.4f} | loss={best_loss:.4f} | val_loss={best_vloss:.4f}")
    print(f"  gap(train-val) en BEST: {gap_best:.4f}")

    print("\n— En ÚLTIMO epoch entrenado (solo para ver tendencia) —")
    print(f"  acc={last_acc:.4f} | val_acc={last_vacc:.4f} | loss={last_loss:.4f} | val_loss={last_vloss:.4f}")
    print(f"  gap(train-val) en ÚLTIMO: {gap_last:.4f}")
    print(f"  slopes últimos 5: acc_tr={s_acc:.4f}, acc_val={s_vacc:.4f}, loss_tr={s_loss:.4f}, loss_val={s_vloss:.4f}")

    near_chance = chance + 0.03

    if not np.isnan(best_vacc) and best_vacc <= near_chance:
        print("\n⚠️ VALIDACIÓN CERCA DE AZAR (pipeline/labels/split sospechoso)")
        print("Acciones: revisar DATA_DIR, clases, etiquetas, y que train/val/test tengan todas las clases.")
        return

    if (not np.isnan(best_acc) and best_acc < 0.60) and (not np.isnan(best_vacc) and best_vacc < 0.60):
        print("\n🟡 SUBAPRENDIZAJE (UNDERFITTING)")
        print("Acciones: más capacidad, más epochs, revisar representación/IMG_SIZE, LR, etc.")
        return

    if (degrade_loss or degrade_acc) and (not np.isnan(gap_best) and gap_best >= 0.12):
        print("\n🔴 OVERFITTING (MEMORIZACIÓN) DESPUÉS DEL BEST")
        print("Señal: el modelo mejoró y luego empeoró en validación.")
        print("Recomendaciones:")
        print("  - Quédate con el modelo del best_epoch (ya queda restaurado si restore_best_weights=True).")
        print("  - Baja patience (p.ej. 3–4) o limita epochs.")
        print("  - Aumenta augment y/o sube dropout.")
        print("  - Si hay duplicados muy parecidos: split por grupo (escena/personaje/objeto).")
        return

    if (not np.isnan(best_vacc) and best_vacc > chance + 0.20) and (not np.isnan(gap_best) and gap_best <= 0.12):
        print("\n✅ TODO BIEN / GENERALIZA RAZONABLEMENTE")
        print("Recomendaciones leves: afinar LR, scheduler, o un poco más de capacidad.")
        return

    print("\n🟢 MIXTO (pero NO roto): aprende, con margen de mejora")
    print("Sugerencias:")
    print("  - Si gap es alto: más regularización/augment o stopping más agresivo.")
    print("  - Si val se estanca: LR menor o scheduler.")
    print("Nota: si restore_best_weights=True, el modelo final es el del best_epoch.")

diagnose_training_v2(history, num_classes=num_classes, monitor_metric=MONITOR_METRIC, patience=PATIENCE)



DIAGNÓSTICO 7.5 — RESUMEN (v2)
Clases: 10 | azar≈ 0.1000 | epochs corridos: 24
Monitor: val_accuracy (max) | best_epoch=19 | best=0.8835
Patience: 5

— En BEST epoch (lo que queda en memoria si restore_best_weights=True) —
  acc=0.8863 | val_acc=0.8835 | loss=0.3091 | val_loss=0.3325
  gap(train-val) en BEST: 0.0028

— En ÚLTIMO epoch entrenado (solo para ver tendencia) —
  acc=0.8913 | val_acc=0.8019 | loss=0.2919 | val_loss=0.5656
  gap(train-val) en ÚLTIMO: 0.0894
  slopes últimos 5: acc_tr=0.0009, acc_val=0.0040, loss_tr=-0.0029, loss_val=-0.0225

✅ TODO BIEN / GENERALIZA RAZONABLEMENTE
Recomendaciones leves: afinar LR, scheduler, o un poco más de capacidad.


In [12]:
# ==========================================================
# CELDA 8 — EVALUACIÓN EN TEST
# ==========================================================
test_out = model.evaluate(test_ds, verbose=0)
print("TEST metrics:")
for name, val in zip(model.metrics_names, test_out):
    print(f"  {name:>12s}: {val:.4f}")


TEST metrics:
          loss: 0.3376
  compile_metrics: 0.8780


In [13]:
# ==========================================================
# CELDA 9 — REPORTE + MATRIZ DE CONFUSIÓN
# ==========================================================
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_true, y_pred = [], []

for x, y in test_ds:
    p = model.predict(x, verbose=0)
    y_true.extend(y.numpy().tolist())
    y_pred.extend(np.argmax(p, axis=1).tolist())

cm = confusion_matrix(y_true, y_pred)
print("Matriz de confusión shape:", cm.shape)

print("\nClassification report:")
print(classification_report(y_true, y_pred, target_names=classes, digits=4))


Matriz de confusión shape: (10, 10)

Classification report:
              precision    recall  f1-score   support

           0     0.6859    0.9295    0.7893      1050
           1     0.9837    0.9790    0.9814      1050
           2     0.9245    0.7114    0.8041      1050
           3     0.8491    0.9219    0.8840      1050
           4     0.8315    0.8410    0.8362      1050
           5     0.9969    0.9067    0.9496      1050
           6     0.7746    0.6152    0.6858      1050
           7     0.8551    0.9895    0.9174      1050
           8     0.9790    0.9771    0.9781      1050
           9     0.9835    0.9086    0.9446      1050

    accuracy                         0.8780     10500
   macro avg     0.8864    0.8780    0.8770     10500
weighted avg     0.8864    0.8780    0.8770     10500



In [14]:
# ==========================================================
# CELDA 10 — EXPORTAR "resultados.zip" (inferencia reproducible)
# Contiene:
#   - model.keras (modelo completo)
#   - weights.best.keras (pesos del mejor checkpoint, si existe)
#   - metadata.json (config + clases + parámetros)
#   - infer_from_zip.py (script para predecir un ZIP nuevo)
#   - README_INFERENCIA.txt (guía rápida)
# Salida: /content/resultados.zip
# ==========================================================
import os, json, zipfile, shutil, time
import numpy as np
import tensorflow as tf

OUT_ZIP = "/content/resultados.zip"
BUNDLE_DIR = "/content/_bundle_resultados"

# Limpia bundle
if os.path.isdir(BUNDLE_DIR):
    shutil.rmtree(BUNDLE_DIR)
os.makedirs(BUNDLE_DIR, exist_ok=True)

# 1) Guardar modelo completo (incluye arquitectura + pesos actuales en memoria)
MODEL_PATH = os.path.join(BUNDLE_DIR, "model.keras")
model.save(MODEL_PATH)

# 2) Copiar checkpoint best si existe
WEIGHTS_SRC = "/content/weights.best.keras"
WEIGHTS_DST = os.path.join(BUNDLE_DIR, "weights.best.keras")
if os.path.isfile(WEIGHTS_SRC):
    shutil.copy2(WEIGHTS_SRC, WEIGHTS_DST)

# 3) Guardar metadata
meta = {
    "created_at": time.strftime("%Y-%m-%d %H:%M:%S"),
    "zip_train_source": os.path.basename(zip_name) if "zip_name" in globals() else None,
    "data_dir_train": DATA_DIR if "DATA_DIR" in globals() else None,
    "seed": int(SEED),
    "train_frac": float(TRAIN_FRAC),
    "val_frac": float(VAL_FRAC),
    "test_frac": float(TEST_FRAC),
    "auto": bool(AUTO),
    "img_size": [int(IMG_SIZE[0]), int(IMG_SIZE[1])],
    "channels": int(CHANNELS),
    "batch_final": int(BATCH),
    "num_classes": int(num_classes),
    "classes": list(classes),
    "normalize": "x/255.0",
    "decoder": "tf.io.decode_image(channels=CHANNELS, expand_animations=False)",
    "resize": "tf.image.resize(img_size, antialias=True)",
    "prediction": {
        "type": "multiclass",
        "activation": "softmax",
        "label_type": "int index -> classes[index]"
    }
}
with open(os.path.join(BUNDLE_DIR, "metadata.json"), "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

# 4) Script de inferencia desde un ZIP nuevo
infer_py = r'''
import os, glob, zipfile, shutil, json
import numpy as np
import tensorflow as tf

WORKDIR = "/content/new_zip_workdir"
EXTS = (".jpg",".jpeg",".png",".bmp",".webp")

def find_images_root(workdir):
    # Permite: raíz con imágenes, o subcarpetas con imágenes (cualquier profundidad 1)
    candidates = [workdir] + [os.path.join(workdir, d) for d in os.listdir(workdir) if os.path.isdir(os.path.join(workdir, d))]

    def score_dir(d):
        if not os.path.isdir(d): return (-1, -1)
        n = 0
        for ext in EXTS:
            n += len(glob.glob(os.path.join(d, f"*{ext}")))
        # si tiene subcarpetas, suma también imágenes dentro de cada subcarpeta (1 nivel)
        subdirs = [os.path.join(d, s) for s in os.listdir(d) if os.path.isdir(os.path.join(d, s))]
        n2 = 0
        for sd in subdirs:
            for ext in EXTS:
                n2 += len(glob.glob(os.path.join(sd, f"*{ext}")))
        return (n, n2)

    best = None
    best_sc = (-1, -1)
    for c in candidates:
        sc = score_dir(c)
        if sc > best_sc:
            best_sc = sc
            best = c
    if best is None or (best_sc[0] + best_sc[1]) == 0:
        raise ValueError(f"No se encontraron imágenes en {workdir}")
    return best

def list_all_images(root_dir):
    # Si root_dir tiene imágenes directas, usa esas; si no, usa todas las subcarpetas (1 nivel)
    direct = []
    for ext in EXTS:
        direct += glob.glob(os.path.join(root_dir, f"*{ext}"))
    direct = sorted(direct)
    if len(direct) > 0:
        return direct

    files = []
    subdirs = [os.path.join(root_dir, s) for s in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, s))]
    for sd in sorted(subdirs):
        for ext in EXTS:
            files += glob.glob(os.path.join(sd, f"*{ext}"))
    return sorted(files)

def decode_image(path, img_size, channels):
    img = tf.io.read_file(path)
    img = tf.io.decode_image(img, channels=channels, expand_animations=False)
    img = tf.image.resize(img, img_size, antialias=True)
    img = tf.cast(img, tf.float32) / 255.0
    return img

def make_ds(files, img_size, channels, batch):
    ds = tf.data.Dataset.from_tensor_slices(files)
    ds = ds.map(lambda p: decode_image(p, img_size, channels), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds

def predict_zip(zip_path, model_path="model.keras", metadata_path="metadata.json", out_csv="/content/predicciones.csv"):
    # Cargar metadata
    with open(metadata_path, "r", encoding="utf-8") as f:
        meta = json.load(f)
    img_size = tuple(meta["img_size"])
    channels = int(meta["channels"])
    classes = meta["classes"]
    batch = int(meta.get("batch_final", 32))

    # Preparar workdir
    if os.path.isdir(WORKDIR):
        shutil.rmtree(WORKDIR)
    os.makedirs(WORKDIR, exist_ok=True)

    # Extraer zip
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(WORKDIR)

    # Detectar raíz con imágenes
    root = find_images_root(WORKDIR)
    files = list_all_images(root)
    if len(files) == 0:
        raise ValueError("No se encontraron imágenes para predecir.")

    # Cargar modelo
    model = tf.keras.models.load_model(model_path)

    # Predicción
    ds = make_ds(files, img_size, channels, batch)
    probs = model.predict(ds, verbose=0)
    pred_idx = np.argmax(probs, axis=1)
    pred_name = [classes[i] for i in pred_idx]
    conf = np.max(probs, axis=1)

    # Guardar CSV
    import csv
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["filepath", "pred_idx", "pred_class", "confidence"])
        for p, i, c, cf in zip(files, pred_idx, pred_name, conf):
            w.writerow([p, int(i), c, float(cf)])

    print("OK ✅")
    print("ZIP:", zip_path)
    print("Imágenes:", len(files))
    print("Salida CSV:", out_csv)
    return out_csv

if __name__ == "__main__":
    # Usa el ZIP más reciente en /content por defecto
    zips = sorted(glob.glob("/content/*.zip"), key=os.path.getmtime)
    assert len(zips) > 0, "No hay ZIP en /content"
    zip_path = zips[-1]
    predict_zip(zip_path)
'''
with open(os.path.join(BUNDLE_DIR, "infer_from_zip.py"), "w", encoding="utf-8") as f:
    f.write(infer_py.strip() + "\n")

# 5) README mínimo
readme = f"""\
RESULTADOS — INFERENCIA DESDE UN ZIP NUEVO (IMÁGENES)

Archivos:
- model.keras            : modelo completo (arquitectura + pesos en memoria)
- weights.best.keras     : checkpoint best (si existe)
- metadata.json          : IMG_SIZE, CHANNELS, classes, etc.
- infer_from_zip.py      : script de predicción para un ZIP nuevo

USO EN COLAB:
1) Sube resultados.zip a /content y descomprímelo:
   !unzip -o /content/resultados.zip -d /content/resultados

2) Sube un ZIP NUEVO de imágenes a /content (ej: nuevo.zip)

3) Corre:
   %cd /content/resultados
   !python infer_from_zip.py

Salida:
- /content/predicciones.csv
"""
with open(os.path.join(BUNDLE_DIR, "README_INFERENCIA.txt"), "w", encoding="utf-8") as f:
    f.write(readme)

# 6) Empaquetar resultados.zip
if os.path.isfile(OUT_ZIP):
    os.remove(OUT_ZIP)

with zipfile.ZipFile(OUT_ZIP, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for root, _, files in os.walk(BUNDLE_DIR):
        for fn in files:
            abs_path = os.path.join(root, fn)
            rel_path = os.path.relpath(abs_path, BUNDLE_DIR)
            z.write(abs_path, rel_path)

print("✅ Creado:", OUT_ZIP)
!ls -lah /content/resultados.zip


✅ Creado: /content/resultados.zip
-rw-r--r-- 1 root root 4.6M Jan 21 07:27 /content/resultados.zip
